In [5]:
## import time
import pymongo
import json
import pprint
import re
from pymongo.errors import WriteError, DuplicateKeyError
from pymongo import MongoClient
import datetime
import os
client = MongoClient('mongodb://localhost:27017')
testDB = client.abaco

## Search

In [184]:
# Example request
#'?search=car&exactsearch=DEV-DEVELOP&search=dog&&skip=0&limit=10'

# Mongo aggregation comparison expression operators to implement:
# $eq, $gt, %gte, $lt, $lte, $ne, $in, $nin

def get(search_type, args):
    """
    Placeholder
    """
    #args = dict(args.lists())    
    search_type = search_type.lower()
    queried_store, security, project = get_db_specific_sections(search_type)
    search, query, paging = arg_parser(args, tenant)
    pipeline = search + query + security + paging #project
    pprint.pprint(list(queried_store.aggregate(pipeline))[2:3])
    
        
def get_db_specific_sections(search_type):
    """
    Placeholder
    """
    store_dict = {'executions': testDB['3'],
                  'workers': testDB['6'],
                  'actors': testDB['5'],
                  'logs': testDB['1']}
    try:
        queried_store = store_dict[search_type]
    except KeyError:
        raise KeyError(f'Inputted search_type is invalid, must be one of {list(store_dict.keys())}.')
    
    localField = 'actor_id'
    if search_type =='actors':
        localField = '_id'
    
    security = [{'$match': {'tenant': tenant}},
                {'$lookup':
                    {'from' : '2',
                     'localField' : localField,
                     'foreignField' : '_id',
                     'as' : 'permissions'}},
                {'$unwind': '$permissions'},
                {'$match': {'permissions.' + curr_user: {'$exists': True}}}]
    
    if search_type == 'executions':
        project = [{'project': {'ijaiosjdioasjd'}}]
        
    elif search_type == 'workers':
        project = [{'project': {'ijaiosjdioasjd'}}]
        
    elif search_type == 'actors':
        project = [{'project': {'ijaiosjdioasjd'}}]
        
    elif search_type == 'logs':
        project = [{'project': {'ijaiosjdioasjd'}}]
        
    return queried_store, security, project

def arg_parser(args, tenant):
    """
    Placeholder
    """
    query = []
    search = f'"{tenant}"'
    paging = []
    skip_amo = None
    limit_amo = None
    
    for key, val in args.items():
        if key == "search":
            if isinstance(val, list):
                joined_val = ' '.join(val)
                search = search + f' {joined_val}'
            else:
                search = search + f' {val}'
        elif key == "exactsearch":
            if isinstance(val, list):
                joined_val = '" "'.join(val)
                search = search + f' "{joined_val}"'
            else:
                search = search + f' "{val}"'
        elif key == "skip":
            if isinstance(val, list):
                raise KeyError('Inputted "skip" paramater is invalid, parameter can only be specified once')
            else:
                skip_amo = int(val)
                paging = paging + [{'$skip': skip_amo}]
        elif key == "limit":
            if isinstance(val, list):
                raise KeyError('Inputted "limit" paramater is invalid, parameter can only be specified once')
            else:
                limit_amo = int(val)
                paging = paging + [{'$limit': limit_amo}]
        else:
            used_oper = False
            for oper in ['.$eq', '.$gt', '.$gte', '.$lt', '.$lte', '.$ne']:
                if oper in key:
                    key = key.split(oper)[0]
                    oper = oper[1:]
                    query += [{'$match': {key: {oper: val}}}]
                    used_oper = True
                    break
            if not used_oper:
                query += [{'$match': {key: val}}]

    search = [{'$match': {'$text': {'$search': search}}},
              {'$sort': {'score': {'$meta': 'textScore'}}}]    

    return search, query, paging

In [8]:
import pytz 
AUS = pytz.timezone("America/Chicago")

In [52]:
def broad_ISO_to_datetime(dt_str):
    """
    Parses a broadly conforming ISO 8601 string.
    """
    dt_str = dt_str.replace('Z', '')
    dt_tz_ready = dt_str[:19] + dt_str[19:].replace(':', '')
    datetime_strs = ['%Y-%m-%dT%H:%M:%S.%f%z',
                     '%Y-%m-%dT%H:%M:%S.%f',
                     '%Y-%m-%dT%H:%M:%S%z',
                     '%Y-%m-%dT%H:%M:%S',
                     '%Y-%m-%dT%H:%M%z',
                     '%Y-%m-%dT%H:%M',
                     '%Y-%m-%dT%H%z',
                     '%Y-%m-%dT%H',
                     '%Y-%m-%dT%z',
                     '%Y-%m-%dT',
                     '%Y-%m-%d%z',
                     '%Y-%m-%d',
                     '%Y-%m%z',
                     '%Y-%m',
                     '%Y%z',
                     '%Y']
    # Try each parse for each datetime str
    for datetime_try in datetime_strs:
        # Changing the datetime string to check depending on what we're
        # attempted to parse with datetime.datetime.strptime()
        if datetime_try == '%Y-%m-%dT%H:%M%z':
            dt_tz_ready = dt_str[:16] + dt_str[16:].replace(':', '')
        elif datetime_try ==  '%Y-%m-%dT%H%z':
            dt_tz_ready = dt_str[:13] + dt_str[13:].replace(':', '')
        elif datetime_try == '%Y-%m-%dT%':
            dt_tz_ready = dt_str.replace(':', '')
        try:
            dt = datetime.datetime.strptime(dt_tz_ready, datetime_try)
            return dt
        except:
            pass
    # If no parse was successful, error!
    raise ValueError("Inputted datetime was in an incorrect format."\
                     " Please refer to docs and follow ISO 8601 formatting."\
                     " e.g. '2020-05-01T14:45:41.591Z'")

In [11]:
def broad_ISO_to_datetime(dt_str):
    """

    """
    # There are dz_tz_ready variables to get rid of any colons in timezone information
    # being given in the ISO 8601 format. 
    dt_str = dt_str.replace('Z', '')
    try:
        dt_tz_ready = dt_str[:19] + dt_str[19:].replace(':', '')
        dt = datetime.datetime.strptime(dt_tz_ready, "%Y-%m-%dT%H:%M:%S.%f%z")
    except ValueError:
        try:
            dt = datetime.datetime.strptime(dt_tz_ready, "%Y-%m-%dT%H:%M:%S.%f")
        except ValueError:
            try:
                dt = datetime.datetime.strptime(dt_tz_ready, "%Y-%m-%dT%H:%M:%S%z")
            except ValueError:
                try:
                    dt = datetime.datetime.strptime(dt_tz_ready, "%Y-%m-%dT%H:%M:%S")
                except ValueError:
                    try:
                        dt_tz_ready = dt_str[:16] + dt_str[16:].replace(':', '')
                        dt = datetime.datetime.strptime(dt_tz_ready, "%Y-%m-%dT%H:%M%z")
                    except ValueError:
                        try:
                            dt = datetime.datetime.strptime(dt_tz_ready, "%Y-%m-%dT%H:%M")
                        except ValueError:
                            try:
                                dt_tz_ready = dt_str[:13] + dt_str[13:].replace(':', '')
                                dt = datetime.datetime.strptime(dt_tz_ready, "%Y-%m-%dT%H%z")
                            except:
                                try:
                                    dt = datetime.datetime.strptime(dt_tz_ready, "%Y-%m-%dT%H")
                                except ValueError:
                                    try:
                                        dt_tz_ready = dt_str.replace(':', '')
                                        dt = datetime.datetime.strptime(dt_tz_ready, "%Y-%m-%dT%z") 
                                    except ValueError:
                                        try:
                                            dt = datetime.datetime.strptime(dt_tz_ready, "%Y-%m-%dT")
                                        except ValueError:
                                            try:
                                                dt = datetime.datetime.strptime(dt_tz_ready, "%Y-%m-%d%z")
                                            except ValueError:
                                                try:
                                                    dt = datetime.datetime.strptime(dt_tz_ready, "%Y-%m-%d")
                                                except ValueError:
                                                    try:
                                                        dt = datetime.datetime.strptime(dt_tz_ready, "%Y-%m%z")
                                                    except ValueError:
                                                        try:
                                                            dt = datetime.datetime.strptime(dt_tz_ready, "%Y-%m")
                                                        except ValueError:
                                                            try:
                                                                dt = datetime.datetime.strptime(dt_tz_ready, "%Y%z")
                                                            except ValueError:
                                                                try:
                                                                    dt = datetime.datetime.strptime(dt_tz_ready, "%Y")
                                                                except ValueError:
                                                                    raise ValueError("Inputted datetime was in an incorrect format.",
                                                                                     " Please refer to docs and follow ISO 8601 formatting.",
                                                                                     " e.g. '2020-05-01T14:45:41.591Z'")
    return dt

In [51]:
broad_ISO_to_datetime('dsadas')

ValueError: Inputted datetime was in an incorrect format. Please refer to docs and follow ISO 8601 formatting. e.g. '2020-05-01T14:45:41.591Z'

In [47]:
broad_ISO_to_datetime(datetime.datetime.now(tz=AUS).isoformat())

datetime.datetime(2020, 5, 5, 18, 1, 2, 668361, tzinfo=datetime.timezone(datetime.timedelta(-1, 68400)))

In [13]:
dt_str = datetime.datetime.now(tz=AUS).isoformat()

In [25]:
def camel_to_under(value):
    return re.sub(r'(?<!^)(?=[A-Z])', '_', value).lower()

def dict_to_under(d):
    """Convert all keys in a dictionary to camel case."""
    d2 = {}
    for k,v in d.items():
        k = k.split(".")
        k[0] = camel_to_under(k[0])
        k = ".".join(k)
        d2[k] = v
    return d2

In [187]:
def arg_parser(args):
    query = []
    search = f'"DEV-DEVELOP"'
    skip_amo = 0
    limit_amo = 10

    #case = Config.get('web', 'case')
    case = "camel"
    if case == 'camel':
        args = dict_to_under(args)

    for key, val in args.items():
        if key == "search":
            if isinstance(val, list):
                joined_val = ' '.join(val)
                search = search + f' {joined_val}'
            else:
                search = search + f' {val}'
        elif key == "exactsearch":
            if isinstance(val, list):
                joined_val = '" "'.join(val)
                search = search + f' "{joined_val}"'
            else:
                search = search + f' "{val}"'
        elif key == "skip" or key == "limit":
            try:
                val = int(val)
            except ValueError:
                raise ValueError(f'Inputted "{key}" paramater must be an int. Received: {val}')
            if val < 0:
                raise ValueError(f'Inputted "{key}" must be positive. Received: {val}')
            if key == "skip":
                skip_amo = val
            if key == "limit":
                limit_amo = val
        else:
            time_keys = ['start_time', 'message_received_time', 'last_execution_time',
                        'last_update_time', 'StartedAt', 'FinishedAt', 'create_time',
                        'last_health_check_time']
            if val.lower() == 'false':
                val = False
            elif val.lower() == 'true':
                val = True
            elif val.lower() == 'none':
                val = None
            elif any(time_key in key for time_key in time_keys):
                if '.between' in key:
                    if not ',' in val:
                        raise ValueError('Between must have two variables seperated by a comma. Ex. io.between=20,40')
                    time1, time2 = val.split(',')
                    time1 = broad_ISO_to_datetime(time1)
                    time2 = broad_ISO_to_datetime(time2)
                    val = [time1, time2]
                else:
                    val = broad_ISO_to_datetime(val)
            else:
                try: 
                    val = float(val)
                except ValueError:
                    pass
                except TypeError:
                    pass
            used_oper = False

            oper_aliases = {'.neq': '$ne', '.eq': '$eq', '.lte': '$lte', '.lt': '$lt',
                            '.gte': '$gte', '.gt': '$gt', '.nin': '$nin', '.in': '$in'}
            for oper_alias, mongo_oper in oper_aliases.items():
                if oper_alias in key:
                    key = key.split(oper_alias)[0]
                    query += [{'$match': {key: {mongo_oper: val}}}]
                    used_oper = True
                    break

            if not used_oper:
                if '.between' in key:
                    if isinstance(val, list):
                        if isinstance(val[0], datetime.datetime) and isinstance(val[1], datetime.datetime):
                            key = key.split('.between')[0]
                            query += [{'$match': {key: {'$gte': val[0], '$lte': val[1]}}}]
                        else:
                            raise ValueError("The values given to .between should be either float or ISO 8601 datetime.")
                    else:
                        if not ',' in val:
                            raise ValueError('Between must have two variables seperated by a comma. Ex. io.between=20,40')
                        key = key.split('.between')[0]
                        val1, val2 = val.split(',')
                        try:
                            val1 = float(val1)
                            val2 = float(val2)
                        except ValueError:
                            raise ValueError("The values given to .between should be either float or ISO 8601 datetime.")
                        query += [{'$match': {key: {'$gte': val1, '$lte': val2}}}]
                    
                elif '.nlike' in key:
                    key = key.split('.nlike')[0]
                    query += [{'$match': {key: {'$not': {'$regex': f"{val}"}}}}]

                elif '.like' in key:
                    key = key.split('.like')[0]
                    query += [{'$match': {key: {'$regex': f"{val}"}}}]
 
                else:
                    query += [{'$match': {key: val}}]
                
    search = [{'$match': {'$text': {'$search': search}}},
              {'$sort': {'score': {'$meta': 'textScore'}}}]
    return search, query, skip_amo, limit_amo

In [209]:
tenant = 'DEV-DEVELOP'
curr_user = 'testuser'
# '?search=7KEaNE8lrRw5k&search=KUBS&exactsearch=car&exactsearch=POPS&aid=FORDHmongENRY&aid.$gt=100010010'
# args = {'search': ['7KEaNE8lrRw5k', 'KUBS'], 'exactsearch': ['car', 'POPS'], 'aid':'FORDHENRY', 'aid.$gt': '100010010'}

# '?search=DEV-DEVELOP&aid=FORDHENRY&aid.$gt=100010010&skip=0&limit=10'
# args = {'search': 'DEV-DEVELOP', 'aid':'FORDHENRY', 'aid.$gt': '100010010', 'skip':'0', 'limit':'10'}

args = [{'$match': {'cpu' :{'$in': [433607460]}}}]

#args = {'search': 'DEV-DEVELOP_jJqjmq57Vjv55'}

#args = {'exactsearch': ''}

#args = {'search': 'DEV-DEVELOP', 'actor_id': 'DEV-DEVELOP_jJqjmq57Vjv55'}


#pipeline = arg_parser(args)[1]

In [181]:
pipeline

[{'$match': {'start_time': {'$lt': datetime.datetime(2020, 4, 26, 0, 0)}}}]

In [211]:
list("['car', 'dog']")

['[', "'", 'c', 'a', 'r', "'", ',', ' ', "'", 'd', 'o', 'g', "'", ']']

In [195]:
args

[{'$match': {'executor.nin': ['testuser']}}]

In [210]:
list(testDB['3'].aggregate(args))

[{'_id': 'DEV-DEVELOP_AKxNjP73pqBzw_jjjrG8JDP5emK',
  'actor_id': 'DEV-DEVELOP_AKxNjP73pqBzw',
  'api_server': 'https://dev.tenants.aloedev.tacc.cloud',
  'cpu': 433607460,
  'executor': 'testuser',
  'exit_code': 0,
  'final_state': {'Status': 'exited',
   'Running': False,
   'Paused': False,
   'Restarting': False,
   'OOMKilled': False,
   'Dead': False,
   'Pid': 0,
   'ExitCode': 0,
   'Error': '',
   'StartedAt': datetime.datetime(2020, 4, 29, 19, 18, 26, 557000),
   'FinishedAt': datetime.datetime(2020, 4, 29, 19, 18, 28, 560000)},
  'id': 'jjjrG8JDP5emK',
  'io': 716,
  'message_received_time': datetime.datetime(2020, 4, 29, 19, 18, 25, 820000),
  'runtime': 2,
  'start_time': datetime.datetime(2020, 4, 29, 19, 18, 26, 105000),
  'status': 'COMPLETE',
  'tenant': 'DEV-DEVELOP',
  'worker_id': '4QA8Z6bKKmlJ1'}]

## Indexing stuff

In [24]:
# Do this to create a wildcard index on littttterally everything.
testDB['6'].create_index([('$**', pymongo.TEXT)])

'$**_text'

In [21]:
list(testDB['6'].list_indexes())

[SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_'), ('ns', 'abaco.6')]),
 SON([('v', 2), ('key', SON([('_fts', 'text'), ('_ftsx', 1)])), ('name', '$**_text'), ('ns', 'abaco.6'), ('weights', SON([('$**', 1)])), ('default_language', 'english'), ('language_override', 'language'), ('textIndexVersion', 3)])]

## Basic pipeline

In [31]:
pipeline = [{'$match': {'runtime': 2}}]
list(testDB['3'].aggregate(pipeline))

[]

## Complexer pipeline

In [ ]:
#ex. Update logs fields if actors exists
pipeline = [{'$match': {'$text': {'$search': search}}},
            {'$sort': {'score': {'$meta': 'textScore'}}},
            {'$match': {'tenant': tenant}},
            {'$lookup':
                {'from' : '2',
                 'localField' : 'actor_id',
                 'foreignField' : '_ifd',
                 'as' : 'permissions'}},
            {'$unwind': '$permissions'},
            {'$match': {'permissions.' + curr_user: {'$exists': True}}},
            {'$skip': skip_amo},
            {'$limit': limit_amo}]
            #{'project': {ijaiosjdioasjd}}

db = '2' 
    
pipeline = [{'$lookup':
                {'from' : '5', #Actors_db
                 'localField' : '_id',
                 'foreignField' : '_id',
                 'as' : 'actor_dict'}},
            {'$unwind': '$actor_dict'},
           ]

list(testDB['3'].aggregate(pipeline))

## Complex executions pipeline

In [3]:
def converting_first_result(first_stage_json):
    newList = []
    for actor in first_stage_json:
        list_of_execution_info = []
        aid = actor['_id']
        newDict = {'_id': aid}
        del actor['_id']
        try:
            del actor['permissions']
        except:
            pass
        for key, value in actor.items():
            list_of_execution_info.append(value)
        newDict.update({'executions': list_of_execution_info})
        newList.append(newDict)
    return newList

In [4]:
tenant = 'DEV-DEVELOP'
curr_user = 'testuser'

In [20]:
search = f'DEV-DEVELOP'
pipeline = [{'$match': {'$text': {'$search': search}}},
            {'$sort': {'score': {'$meta': 'textScore'}}}]
old_json = list(testDB['5'].aggregate(pipeline))

In [18]:
old_json

[]

In [5]:
# Executions
# First Pipeline
search = '"DEV-DEVELOP_1952559"'
pipeline = [{'$match': {'$text': {'$search': search}}},
            {'$sort': {'score': {'$meta': 'textScore'}}}]
old_json = list(testDB['3'].aggregate(pipeline))

In [6]:
second_stage_input = converting_first_result(old_json)

In [ ]:
testDB['20'].drop()
start = time.time()

testDB['20'].insert_many(second_stage_input)
print(time.time() - start)

In [ ]:
import time
start = time.time()
pipeline = [{'$match': {'executions.id': '8864362'}}]

old_json = list(testDB['3'].aggregate(pipeline))
print(time.time() - start)


In [ ]:
# Executions
# First Pipeline
search = '"DEV-DEVELOP_1952559"'
pipeline = [{'$match': {'$text': {'$search': search}}},
            {'$sort': {'score': {'$meta': 'textScore'}}}]
old_json = list(testDB['3'].aggregate(pipeline))

second_stage_input = converting_first_result(old_json)

testDB['20'].drop()
start = time.time()

testDB['20'].insert_many(second_stage_input)
print(time.time() - start)

import time
start = time.time()
pipeline = [{'$match': {'executions.id': '8864362'}}]

old_json = list(testDB['3'].aggregate(pipeline))
print(time.time() - start)


## Creating a very large executions store

In [7]:
import random
actorsList = []
for actors in range(2000):
    aid = 'DEV-DEVELOP_' + str(random.randint(1000000, 9999999))
    executionsList = []
    for execs in range(random.randint(100, 2000)):
        wid = random.randint(1000000, 9999999)
        eid = random.randint(1000000, 9999999)
        execDict = {'tenant': 'DEV-DEVELOP',
                    'api_server': 'https://dev.tenants.develop.tacc.cloud',
                    'actor_id': aid,
                    'executor': 'testuser',
                    'worker_id': wid,
                    'id': eid,
                    'message_received_time': '1585583279.07623',
                    'start_time': '1585583279.400957',
                    'runtime': 1,
                    'cpu': 239084680,
                    'io': 180,
                    'status': 'COMPLETE',
                    'exit_code': 0,
                    'final_state': {'Status': 'exited',
                                    'Running': False,
                                    'Paused': False,
                                    'Restarting': False,
                                    'OOMKilled': False,
                                    'Dead': False,
                                    'Pid': 0,
                                    'ExitCode': 0,
                                    'Error': '',
                                    'StartedAt': '2020-03-30T15:47:59.9001718Z',
                                    'FinishedAt': '2020-03-30T15:48:00.654149Z'}}
        executionsList.append(execDict)
    actorsDict = {'_id': aid,
                 'executions': executionsList}
    actorsList.append(actorsDict)
        
expandedList = second_stage_input
expandedList.extend(actorsList)
testDB['3'].drop()

In [8]:
testDB['3'].insert_many(expandedList)

## Archive

In [1]:
# Example request
#'?search=car&exactsearch=DEV-DEVELOP&search=dog&&skip=0&limit=10'

# Mongo aggregation comparison expression operators to implement:
# $eq, $gt, %gte, $lt, $lte, $ne, $in, $nin

def get(search_type, args):
    """
    Placeholder
    """
    # Given 
    tenant = 'DEV-DEVELOP'
    curr_user = 'testuser'
    
    search_type = search_type.lower()
    search, query, paging = arg_parser(args, tenant)
    
    # Executions and Workers store
    if search_type in ['executions', 'workers']:
        nested_search_pipeline(search_type, search, query, paging)
    elif search_type in ['actors', 'logs']:
        flat_search_pipeline(search_type, search, query, paging)
    else:
        raise KeyError(f'Inputted search_type is invalid, must be one of {list(store_dict.keys())}.')
    

def nested_search_pipeline(search_type, search, query, paging):
    store_dict = {'executions': testDB['3'],
                  'workers': testDB['6']}
    queried_store = store_dict[search_type]
        
    # First pipeline setup and work
    security = [{'$lookup':
                    {'from' : '2',
                     'localField' : '_id',
                     'foreignField' : '_id',
                     'as' : 'permissions'}},
                {'$unwind': '$permissions'},
                {'$match': {'permissions.' + curr_user: {'$exists': True}}}]
    first_pipeline = search + security
    first_res = list(queried_store.aggregate(pipeline))
    print('-----PIPELINE-----\n')
    pprint.pprint(first_pipeline)
    print('\n\n\n-----RESULTANT-----')
    pprint.pprint(first_res)

    # Second pipeline setup and work
    flat_first_res = mongo_flattener(first_res)
    ## Should implement a call to use a random/unused database.
    testDB['20'].drop()
    testDB['20'].insert_many(flat_first_res)
    testDB['20'].drop()
    
    # This should be implemented in arg_parser later
    search = [{'$match': {'executions.id': '8864362'}}]
    
    second_pipeline = search + query + security + project + paging
    second_res = list(queried_store.aggregate(pipeline))
    print('-----PIPELINE-----\n')
    pprint.pprint(second_pipeline)
    print('\n\n\n-----RESULTANT-----')
    pprint.pprint(second_res)

    
def arg_parser(args, tenant):
    """
    Placeholder
    """
    query = []
    search_terms = f"{tenant}"
    skip_amo = None
    limit_amo = None
    
    # Parsing args given by user, if multiple given, the arg comes as a list
    for key, val in args.items():
        # Adding search terms to 'search_terms'
        if key == "search":
            if isinstance(val, list):
                joined_val = ' '.join(val)
                search_terms = search_terms + f' {joined_val}'
            else:
                search_terms = search_terms + f' {val}'
        # Adding exact search terms to 'search_terms', but now with double quotes
        elif key == "exactsearch":
            if isinstance(val, list):
                joined_val = '" "'.join(val)
                search_terms = search_terms + f' "{joined_val}"'
            else:
                search_terms = search_terms + f' "{val}"'
        # Gets the number of results to skip
        elif key == "skip":
            if isinstance(val, list):
                raise KeyError('Inputted "skip" paramater is invalid, parameter can only be specified once')
            else:
                skip_amo = int(val)
        # Gets the number of results to show
        elif key == "limit":
            if isinstance(val, list):
                raise KeyError('Inputted "limit" paramater is invalid, parameter can only be specified once')
            else:
                limit_amo = int(val)
        else:
            # Checking for logical operators and adds a pipeline match if found
            used_oper = False
            for oper in ['.$eq', '.$gt', '.$gte', '.$lt', '.$lte', '.$ne']:
                if oper in key:
                    key = key.split(oper)[0]
                    oper = oper[1:]
                    query += [{'$match': {key: {oper: val}}}]
                    used_oper = True
                    break
            # If no local operators found then a regular pipeline match is used
            if not used_oper:
                query += [{'$match': {key: val}}]

    ####### Check if search should start with tenant or not for speed. If so, delete this.
    if search_terms:
        search = [{'$match': {'$text': {'$search': search_terms}}},
                  {'$sort': {'score': {'$meta': 'textScore'}}}]
    #pprint.pprint(search)
    
    paging = []
    if skip_amo:
        paging = paging + [{'$skip': skip_amo}]
    if limit_amo:
        paging = paging + [{'$limit': limit_amo}]
    #pprint.pprint(paging)

    return search, query, paging

def mongo_flattener(mongo_input):
    flatList = []
    for actor in mongo_input:
        list_of_execution_info = []
        aid = actor['_id']
        newDict = {'_id': aid}
        del actor['_id']
        
        ################################################################DELETE THIS LATER
        try:
            del actor['permissions']
        except:
            pass
        for _, value in actor.items():
            list_of_execution_info.append(value)
        newDict.update({'executions': list_of_execution_info})
        flatList.append(newDict)
    return flatList